<a href="https://colab.research.google.com/github/gsc16/MachineLearning/blob/master/hw1q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [1]:
import keras
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical
import math

Using TensorFlow backend.


Loading images from MNIST dataset

In [0]:
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

Displaying the original values

In [3]:
train_images_original.shape, train_labels_original, test_images_original.shape, test_labels_original.shape

((60000, 28, 28),
 array([5, 0, 4, ..., 5, 6, 8], dtype=uint8),
 (10000, 28, 28),
 (10000,))

Extracting images - Reshaping and flattening

In [0]:
train_images_new = train_images_original.reshape(60000, 28*28)
train_images_new = train_images_new.astype('float32') / 255;
train_images_new = train_images_new.T

test_images_new = test_images_original.reshape(10000, 28 *28)   
test_images_new = test_images_new.astype('float32') / 255;
test_images_new = test_images_new.T

train_labels_original = train_labels_original.T
test_labels_original = test_labels_original.T

Converting integer vectors of labels to binary

In [0]:
train_labels_new = keras.utils.to_categorical(train_labels_original, 10)
test_labels_new = keras.utils.to_categorical(test_labels_original, 10)

Displaying new values

In [6]:
train_images_new.shape, train_labels_new.shape, test_images_new.shape, test_labels_new.shape

((784, 60000), (60000, 10), (784, 10000), (10000, 10))

Initializing the learning rate, batch size and number of epochs

In [0]:
learning_rate = 0.001
batch_size = 32
epochs = 12
num_classes =1

Definitions

In [0]:
#Function to calculate logistic regression
def log_reg(w,b,X):
  return np.dot(w.T, X) + b  

# Function to calculate value of sigmoid function
def sigmoid(z):
  return 1.0/(1.0+np.exp(-z))



# Function to calculate the mini batches
def cal_mini_batches(X, Y): 
    for index in range(0, X.shape[0] - batch_size + 1, batch_size):
        new_batch = slice(index, index + batch_size)
        yield X[new_batch], Y[new_batch]
        
# Function to calculate the minibatch stochastic gradient descent - Binary Cross Entropy Loss Function
# This function updates the values of weight and bias based on the learning rate and gradient calculated from the loss function
def minibatch(w, b, X, Y):
  for i in range(epochs):
    for batch in cal_mini_batches(X.T, Y.T):
      x_batch, y_batch = batch
      xt = x_batch.T
      yt = y_batch.T
      l_reg = log_reg(w,b,xt)
      Z = sigmoid(l_reg)
      #sig_prime = sigmoid_prime(Z)
      loss = (Z-yt)
  
      # Calculate the gradients
      gradient_w = np.dot(xt, loss.T)
      gradient_b = np.sum(loss)
      
      w = w - learning_rate * gradient_w 
      b = b - learning_rate * gradient_b
  return w,b


#Function for calcukating the prediction accuracy of images: 
#This function returns 1 if the sigmoid value of images is greater than 0.5(i.e., the digit classified is same as the label), else it returns 0
def cal_prediction(w, b, X):
  pred = np.zeros((1, X.shape[1])) 
  w = w.reshape(X.shape[0], num_classes)
  l_reg = log_reg(w,b,X)
  Z = sigmoid(l_reg)
  
  for i in range(Z.shape[1]):
    pred[0, i] = 1 if Z[0, i] > 0.5 else 0 # storing 0 or 1 based on the confidence value of the predicted output
  
  return pred
        

In [0]:
# Function to run the binary classification model for each classifier(0 to 9)
# This function calculates the test accuracies for each classifier and returns it

def classifier_model(train_images_new, train_labels_original, test_images_new, test_labels_original,train_labels_new,test_labels_new):
 
  training_accuracy = list()
  test_accuracy = list()
  for i in range(0,10):
      # set the digit to classify
      classifier_digit = i;
      
      #modifying training labels to create single class classification
      train_labels_modified = np.array(train_labels_original);
      train_labels_modified = np.where(train_labels_modified == classifier_digit, 1, 0)
      
      test_labels_modified = np.array(test_labels_original);
      test_labels_modified = np.where(test_labels_modified == classifier_digit, 1, 0)
      
      #Initializing the weight vector and bias
      w = np.zeros(shape=(train_images_new.shape[0], 1))
      b = 0
      w , b = minibatch(w, b, train_images_new, train_labels_modified)

      # Predicting the values of the labels based on the given weight and bias
      prediction_train = cal_prediction(w, b, train_images_new)
      prediction_test = cal_prediction(w,b, test_images_new) 
      
      # Create array similar to train_labels_new and test_labels_new to store the categorical values of labels
      train_labels_new[:,[i]] = prediction_train.T
      test_labels_new[:,[i]] = prediction_test.T
      
      # Calculate the training accuracy for each classifier
      train_acc = 100 - np.mean(np.abs(prediction_train - train_labels_modified)) * 100
      training_accuracy.append(train_acc)
      print("Training accuracy of classifer " + str(i) + " is :  "+str(train_acc))
      
      # Calculate the test accuracy for each classifier
      test_acc = 100 - np.mean(np.abs(prediction_test - test_labels_modified)) * 100
      test_accuracy.append(test_acc)
      print("Test accuracy of classifer " + str(i) +" is : " + str(test_acc))
    
  return test_accuracy; 

Training the model for each classifier

In [10]:
ta = classifier_model(train_images_new, train_labels_original, test_images_new, test_labels_original,train_labels_new,test_labels_new)

Training accuracy of classifer 0 is :  99.13
Test accuracy of classifer 0 is : 99.19
Training accuracy of classifer 1 is :  99.08666666666667
Test accuracy of classifer 1 is : 99.33
Training accuracy of classifer 2 is :  97.845
Test accuracy of classifer 2 is : 97.89
Training accuracy of classifer 3 is :  97.53166666666667
Test accuracy of classifer 3 is : 97.81
Training accuracy of classifer 4 is :  98.28833333333333
Test accuracy of classifer 4 is : 98.17
Training accuracy of classifer 5 is :  97.19333333333333
Test accuracy of classifer 5 is : 97.44
Training accuracy of classifer 6 is :  98.73333333333333
Test accuracy of classifer 6 is : 98.68
Training accuracy of classifer 7 is :  98.425
Test accuracy of classifer 7 is : 98.52
Training accuracy of classifer 8 is :  95.92666666666666
Test accuracy of classifer 8 is : 96.11
Training accuracy of classifer 9 is :  96.41
Test accuracy of classifer 9 is : 96.43


Determining the classifier with maximum accuracy

In [11]:
print(ta)
strong_digit = 0
strong_classified = ta[0]
classified = np.argmax(ta)
classified_value = np.argmax(ta)
for i in range(0,len(ta)):
  if ta[i] > strong_classified:
    strong_classified = ta[i]
    strong_digit = i
print("The most strongly classified digit is "+ str(classified) +" with accuracy "+str(strong_classified))

[99.19, 99.33, 97.89, 97.81, 98.17, 97.44, 98.68, 98.52, 96.11, 96.43]
The most strongly classified digit is 1 with accuracy 99.33
